In [1]:
import os
import sys
import glob

import numpy as np
import pandas as pd

## Get TAD data

Download, decompress, and import data from Miami

In [2]:
! mkdir -p ../data/external/tads/
! wget http://dna.cs.miami.edu/TADKB/download/TAD_annotations.tar.gz --directory-prefix=../data/external/tads/
! tar -zxvf ../data/external/tads/TAD_annotations.tar.gz -C ../data/external/tads/

--2020-12-02 21:52:33--  http://dna.cs.miami.edu/TADKB/download/TAD_annotations.tar.gz
Resolving dna.cs.miami.edu (dna.cs.miami.edu)... 192.31.89.34
Connecting to dna.cs.miami.edu (dna.cs.miami.edu)|192.31.89.34|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1473379 (1.4M) [application/x-gzip]
Saving to: ‘../data/external/tads/TAD_annotations.tar.gz.3’

TAD_annotations.tar 100%[===================>]   1.40M  3.27MB/s    in 0.4s    

2020-12-02 21:52:34 (3.27 MB/s) - ‘../data/external/tads/TAD_annotations.tar.gz.3’ saved [1473379/1473379]

TAD_annotations/
TAD_annotations/TADs/
TAD_annotations/TADs/HiC_GM12878_DI_10kb.txt
TAD_annotations/TADs/SPRITE_GM12878_IS_10kb.txt
TAD_annotations/TADs/HiC_HMEC_DI_10kb.txt
TAD_annotations/TADs/HiC_ES_IS_10kb.txt
TAD_annotations/TADs/HiC_HUVEC_DI_10kb.txt
TAD_annotations/TADs/HiC_NPC_IS_10kb.txt
TAD_annotations/TADs/HiC_IMR90_DI_10kb.txt
TAD_annotations/TADs/HiC_CN_IS_10kb.txt
TAD_annotations/TADs/HiC_K562_DI_10kb.txt
TAD_an

In [3]:
TAD_caller = 'GMAP'
TAD_resolution = 10
TAD_fn = '../data/external/tads/TAD_annotations/TADs/HiC_K562_{}_{}kb.txt' \
           .format(TAD_caller, TAD_resolution)

TAD_data = pd.read_csv(TAD_fn, sep=r"\s+", header=None, names=['chrom','start','end'])

In [4]:
TAD_data

,chrom,start,end
0,chr1,705000,3355000
1,chr1,3355000,6735000
2,chr1,6735000,7715000
3,chr1,7715000,8415000
4,chr1,8415000,9645000
5,chr1,9645000,11055000
6,chr1,11055000,11795000
7,chr1,11795000,13765000
8,chr1,13765000,14295000
9,chr1,14295000,15415000


## Get CASA peak calls

In [5]:
! wget https://www.biorxiv.org/content/biorxiv/early/2020/05/12/2020.05.11.078675/DC7/embed/media-7.xlsx --directory-prefix=../data/
! mv ../data/media-7.xlsx ../data/published_peaks.xlsx

--2020-12-02 21:52:34--  https://www.biorxiv.org/content/biorxiv/early/2020/05/12/2020.05.11.078675/DC7/embed/media-7.xlsx
Resolving www.biorxiv.org (www.biorxiv.org)... 104.18.8.18, 104.18.9.18
Connecting to www.biorxiv.org (www.biorxiv.org)|104.18.8.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘../data/media-7.xlsx’

media-7.xlsx            [ <=>                ]  29.02K  --.-KB/s    in 0s      

2020-12-02 21:52:35 (107 MB/s) - ‘../data/media-7.xlsx’ saved [29720]



In [6]:
# Load BioRXiv version
#hcr_peaks = pd.ExcelFile("../data/published_peaks.xlsx").parse(u"Sheet1")
# Load NatGen revisions version
hcr_peaks = pd.ExcelFile("../data/Supplementary Table 5.xlsx").parse(u"Sheet1")

In [7]:
hcr_peaks

,chrom,CRE_start,CRE_end,CRE_activity_summit,target_gene,ROPE_threshold,low_coverage
0,chr11,61813200,61813600,0.926668,FADS1,log(2.00),NaN
1,chr11,61814000,61818400,4.018398,FADS1,log(2.00),NaN
2,chr11,61833100,61835900,4.034463,FADS1,log(2.00),NaN
3,chr11,61841200,61842900,2.039930,FADS1,log(2.00),NaN
4,chr11,61868900,61871800,1.899701,FADS1,log(2.00),NaN
5,chr11,61814500,61817200,2.321921,FADS2,log(1.60),NaN
6,chr11,61833500,61835900,2.161371,FADS2,log(1.60),NaN
7,chr11,61841500,61842400,1.162228,FADS2,log(1.60),NaN
8,chr11,61864500,61864800,1.267389,FADS2,log(1.60),NaN
9,chr11,61816500,61817100,-0.614936,FADS3,log(1.60),NaN


## Get house GFFs for studied loci

In [8]:
my_bucket = 'gs://haddath/sgosai/hff/collate_detailed'
my_loci   = ['MEF', 'NMU', 'CD164', 'ATXN', 'GATA', 'LMO2', 'FADS','MYC']
my_gff_files = [ os.path.join(my_bucket,f'{locus}_locus.gff') for locus in my_loci ]
my_gff = pd.concat([ pd.read_csv(fn, sep=r"\s+", header=0) for fn in my_gff_files ], 
                   ignore_index=True).reset_index(drop=True)


/home/sagergosai/miniconda3/envs/casa/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [9]:
my_gff

,cdsEnd,cdsStart,chrom,exonCount,exonEnds,exonStarts,geneID,name,proteinID,strand,transcriptID,txEnd,txStart
0,88268723,88190051,chr5,13,"88190261,88196488,88197765,88199150,88203075,8...","88189632,88196375,88197668,88198975,88202961,8...",TMEM161B,ENST00000514135.5,E9PCX5,-,uc032vcj.2,88268830,88189632
1,88823788,88722603,chr5,11,"88722925,88728628,88729347,88730234,88731901,8...","88717116,88728492,88729217,88730210,88731728,8...",MEF2C,ENST00000504921.6,Q06413,-,uc003kjj.4,88883464,88717116
2,55425452,55396189,chr4,3,"55396396,55424643,55425657","55395956,55424537,55425375",TMEM165,ENST00000506198.5,D6RD79,+,uc062wsu.1,55425657,55395956
3,55482785,55435414,chr4,23,"55435594,55438537,55442634,55443896,55444785,5...","55427902,55438281,55442431,55443686,55444632,5...",CLOCK,ENST00000513440.6,O15516,-,uc003hba.4,55546909,55427902
4,55592109,55556556,chr4,6,"55556711,55562612,55569861,55580911,55582237,5...","55556518,55562403,55569717,55580820,55582116,5...",PDCL2,ENST00000295645.9,Q8N4E4,-,uc003hbb.4,55592245,55556518
5,55636192,55599145,chr4,10,"55595411,55599181,55600575,55605349,55607348,5...","55595230,55599141,55600521,55605274,55607297,5...",NMU,ENST00000264218.7,P48645,-,uc003hbc.3,55636298,55595230
6,108973514,108876174,chr6,17,"108848546,108854485,108868995,108876350,108894...","108848415,108854224,108868823,108876142,108894...",ARMC2,ENST00000368972.7,Q8NEN0,+,uc011eao.3,108974472,108848415
7,109001458,108987543,chr6,10,"108987630,108988687,108990835,108992899,108994...","108986441,108988542,108990644,108992786,108994...",SESN1,ENST00000302071.6,Q9Y6P5,-,uc021zdp.2,109008988,108986441
8,109155889,109145221,chr6,8,"109095575,109106090,109145381,109146937,109150...","109095109,109106034,109145218,109146757,109150...",CEP57L1,ENST00000521277.5,E5RJH1,+,uc063qmn.1,109158488,109095109
9,109382378,109368850,chr6,6,"109369017,109370467,109376112,109377971,109379...","109366513,109370410,109376073,109377899,109379...",CD164,ENST00000310786.10,Q04900,-,uc003pte.6,109382467,109366513


## Quant fraction of CASA peaks in same TAD as promoter

In [10]:
res_str = 'For gene {}, {}/{} CASA peaks are in the same TAD'

for YFG in hcr_peaks['target_gene'].unique():
    gene_def = my_gff[ my_gff['geneID'] == YFG ].iloc[0]
    promoter = gene_def['txStart'] if gene_def['strand'] == '+' else gene_def['txEnd']
    find_chr = TAD_data['chrom'] == gene_def['chrom']
    find_start=TAD_data['start'] <= promoter
    find_end = TAD_data['end']   >  promoter
    ref_tad  = TAD_data[ find_chr & find_start & find_end ]
    ref_peaks= hcr_peaks[ hcr_peaks['target_gene'] == YFG ]
    find_peak= (ref_peaks['CRE_start'] < ref_tad['end'].item()) & \
               (ref_peaks['CRE_end'] > ref_tad['start'].item())
    print(res_str.format(YFG, sum(find_peak), len(find_peak)))


For gene FADS1, 5/5 CASA peaks are in the same TAD
For gene FADS2, 4/4 CASA peaks are in the same TAD
For gene FADS3, 6/6 CASA peaks are in the same TAD
For gene FEN1, 2/2 CASA peaks are in the same TAD
For gene GATA1, 3/3 CASA peaks are in the same TAD
For gene HDAC6, 3/3 CASA peaks are in the same TAD
For gene NMU, 11/11 CASA peaks are in the same TAD
For gene LMO2, 6/6 CASA peaks are in the same TAD
For gene CAPRIN1, 4/4 CASA peaks are in the same TAD
For gene CAT, 2/2 CASA peaks are in the same TAD
For gene CD164, 5/5 CASA peaks are in the same TAD
For gene MEF2C, 3/3 CASA peaks are in the same TAD
For gene ERP29, 4/4 CASA peaks are in the same TAD
For gene MYC, 5/5 CASA peaks are in the same TAD
For gene PVT1, 13/13 CASA peaks are in the same TAD


In [11]:
res_str = 'For gene {}, {}/{} CASA peaks are in the same TAD'

for YFG in hcr_peaks['target_gene'].unique():
    gene_def = my_gff[ my_gff['geneID'] == YFG ].iloc[0]
    promoter = gene_def['txStart'] if gene_def['strand'] == '+' else gene_def['txEnd']
    find_chr = TAD_data['chrom'] == gene_def['chrom']
    find_start=TAD_data['start'] <= promoter
    find_end = TAD_data['end']   >  promoter
    ref_tad  = TAD_data[ find_chr & find_start & find_end ]
    ref_peaks= hcr_peaks[ hcr_peaks['target_gene'] == YFG ]
    find_peak= (ref_peaks['CRE_start'] < ref_tad['end'].item()) & \
               (ref_peaks['CRE_end'] > ref_tad['start'].item())
    print(res_str.format(YFG, sum(find_peak), len(find_peak)))
    print(ref_tad)
    print(ref_peaks)


For gene FADS1, 5/5 CASA peaks are in the same TAD
      chrom     start       end
1502  chr11  61495000  62655000
   chrom  CRE_start   CRE_end  CRE_activity_summit target_gene ROPE_threshold  \
0  chr11   61813200  61813600             0.926668       FADS1      log(2.00)   
1  chr11   61814000  61818400             4.018398       FADS1      log(2.00)   
2  chr11   61833100  61835900             4.034463       FADS1      log(2.00)   
3  chr11   61841200  61842900             2.039930       FADS1      log(2.00)   
4  chr11   61868900  61871800             1.899701       FADS1      log(2.00)   

   low_coverage  
0           NaN  
1           NaN  
2           NaN  
3           NaN  
4           NaN  
For gene FADS2, 4/4 CASA peaks are in the same TAD
      chrom     start       end
1502  chr11  61495000  62655000
   chrom  CRE_start   CRE_end  CRE_activity_summit target_gene ROPE_threshold  \
5  chr11   61814500  61817200             2.321921       FADS2      log(1.60)   
6  chr11   61